# Part 4: More Colors
So we have a neural network that can, with a lot of repetition in the form of a high epoch count, get pretty good at figuring out three colors. Let's give it a much larger list of colors and see what happens. First we'll simply copy the network from the previous part. Click Run to create the train() function.

In [1]:
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential
import keras.optimizers, keras.utils, numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

def train(colorNameToRGBMap, epochs = 16):
    """
    Trains a neural network to understand how to map color names to RGB triples.
    The provided map is from 'color-name':(r,g,b) where r,g,b are floats in the range [0,1].
    Different names are allowed to map to the same RGB triple.
    Returns a trained model that can be used for recognize().
    """

    # Convert the Python map RGB values into a numpy array needed for training.
    rgbNumpyArray = numpy.array(list(colorNameToRGBMap.values()), numpy.float)
    
    # Convert the color labels into a one-hot feature array.
    # Text labels for each array position are in the classes_ list on the binarizer.
    labelBinarizer = LabelBinarizer()
    oneHotLabels = labelBinarizer.fit_transform(list(colorNameToRGBMap.keys()))
    numColors = len(labelBinarizer.classes_)
    colorLabels = labelBinarizer.classes_
    
    # Hyperparameters to define the network shape.
    numFullyConnectedPerceptrons = numColors * 16
    batchSize = 1
    
    model = Sequential([
        # Layer 1: Fully connected layer with ReLU activation.
        Dense(numFullyConnectedPerceptrons, activation='relu', kernel_initializer='TruncatedNormal', input_shape=(3,)),

        # Outputs: SoftMax activation to get probabilities by color.
        Dense(numColors, activation='softmax')
    ])

    print(model.summary())

    # Compile for categorization.
    model.compile(
        optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9, decay = 1e-6, nesterov = True),
        loss = 'categorical_crossentropy',
        metrics = [ 'accuracy' ])

    history = model.fit(rgbNumpyArray, oneHotLabels, epochs=epochs, batch_size=batchSize)

    return (model, colorLabels)

Using TensorFlow backend.


OK, now we have the train() function, let's train it with a much longer list of colors this time. Click Run.

In [2]:
colorMap = {
    'red': (1.0, 0.0, 0.0),
    'green': (0.0, 1.0, 0.0),
    'blue': (0.0, 0.0, 1.0),
    'yellow': (1.0, 1.0, 0.0),
    'cyan': (0.0, 1.0, 1.0),
    'gray': (0.5, 0.5, 0.5),
    'black': (0.0, 0.0, 0.0),
    'white': (1.0, 1.0, 1.0),
    'magenta': (1.0, 0.0, 1.0),
    'purple': (0.5, 0.0, 1.0),
    'indigo': (0.25, 0.0, 1.0)
}

(colorModel, colorLabels) = train(colorMap)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 176)               704       
_________________________________________________________________
dense_2 (Dense)              (None, 11)                1947      
Total params: 2,651
Trainable params: 2,651
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1/16
11/11 [==============================] - 0s 29ms/step - loss: 2.4430 - acc: 0.0000e+00
Epoch 2/16
11/11 [==============================] - 0s 3ms/step - loss: 2.3938 - acc: 0.0000e+00
Epoch 3/16
11/11 [==============================] - 0s 4ms/step - loss: 2.3250 - acc: 0.1818
Epoch 4/16
11/11 [==============================] - 0s 2ms/step - loss: 2.2779 - acc: 0.3636
Epoch 5/16
11/11 [============

Check out the loss values! With 3 colors we started at about 1.1 and got down to 0.8 after 16 epochs. But with more colors we started with a loss of about 2.4 and got only down to 1.2. The network is having trouble learning even with 16 repetitions.

Let's check out what the color sliders say this time.

In [3]:
from IPython.core.display import display, HTML
from ipywidgets import interact
def displayColor(r, g, b):
    rInt = min(255, max(0, int(r * 255.0)))
    gInt = min(255, max(0, int(g * 255.0)))
    bInt = min(255, max(0, int(b * 255.0)))
    hexColor = "#%02X%02X%02X" % (rInt, gInt, bInt)
    display(HTML('<div style="width: 50%; height: 50px; background: ' + hexColor + ';"></div>'))

@interact(r = (0.0, 1.0, 0.01), g = (0.0, 1.0, 0.01), b = (0.0, 1.0, 0.01))
def getPredictionsFromModel(r, g, b):
    testColor = numpy.array([ (r, g, b) ])
    predictions = colorModel.predict(testColor, verbose=0)  # Predictions shape (1, numColors)
    predictions *= 100.0
    print(colorLabels)
    numpy.set_printoptions(precision=1, suppress=True)
    print(predictions[0])
    displayColor(r, g, b)


interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, step=0.01), FloatSlider(value=0.5, desc…

On my machine the prediction for gray is only 15%, which is quite bad, and in fact it predicts white at 18% which means it's really not trained well at all. Here's the epoch slider again, allowing up to 500 epochs which will take quite awhile. Try different values and see what you get for the loss values and the resulting prediction accuracy.

In [4]:
@interact(epochs = (1, 500))
def trainModel(epochs=10):
    global colorModel
    global colorLabels
    (colorModel, colorLabels) = train(colorMap, epochs=epochs)


interactive(children=(IntSlider(value=10, description='epochs', max=500, min=1), Output()), _dom_classes=('wid…

In [5]:
interact(getPredictionsFromModel, r = (0.0, 1.0, 0.01), g = (0.0, 1.0, 0.01), b = (0.0, 1.0, 0.01))

interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, step=0.01), FloatSlider(value=0.5, desc…

<function __main__.getPredictionsFromModel(r, g, b)>

When I set it to 400 epochs my loss got to around 0.01, at which point the predictions started to get rather good, with the initial gray at 98% and in-between colors coming out well.

### Coming up...
Let's give the network a rather long list of colors to see whether it can handle it.